In [1]:
# import library
import pandas as pd
import numpy as np
import warnings
import math

In [2]:
# import data
warnings.filterwarnings("ignore")
data = pd.read_csv("C:/Users/alexa/Downloads/D483.csv",low_memory=True, encoding='unicode_escape')
data.shape

(483, 82)

In [3]:
data.head()

,TestResult,pub,WhytestedOther,Flutest,symptoms,symptomsresp,symptomsconsresp,symptomsgastro,symptomsconsgastro,symptomsneuro,...,explasttime4=publictrans,explasttime5=airplane,explasttime6=cruise,explasttime7=barcaferest,explasttime8=grocery,explasttime9=retail,explasttime10=visit,explasttime11=gym_team,explasttime12=rlgssvc,explasttime13=hcp
0,"Yes, and I tested negative",I came into contact with someone who has the c...,NaN,"No, I have not been tested",None,None,NaN,None,NaN,None,...,Mid March to May 2020,Mid March to May 2020,Mid March to May 2020,More recently,More recently,More recently,More recently,Mid March to May 2020,More recently,August/September 2020
1,"Yes, and I tested negative",I came into contact with someone who has the c...,NaN,"Yes, and I tested negative",None,None,NaN,None,NaN,None,...,Feb. to early March 2020 or before,Feb. to early March 2020 or before,Never,More recently,More recently,More recently,More recently,Feb. to early March 2020 or before,More recently,August/September 2020
2,"Yes, and I tested positive",I came into contact with someone who has the c...,NaN,"No, I have not been tested",Fatigue (more than normal),Sore throat,No,Diarrhea,No,Headaches,...,Feb. to early March 2020 or before,Feb. to early March 2020 or before,Feb. to early March 2020 or before,More recently,More recently,August/September 2020,More recently,Feb. to early March 2020 or before,August/September 2020,More recently
3,"Yes, and I tested negative",I came into contact with someone who has the c...,NaN,"No, I have not been tested",NaN,"Runny nose,Sore throat",Yes,"Diarrhea,Stomach or abdominal pain",Yes,Headaches,...,Mid March to May 2020,Mid March to May 2020,NaN,August/September 2020,More recently,June/July 2020,August/September 2020,Mid March to May 2020,Mid March to May 2020,August/September 2020
4,"Yes, and I tested negative",I came into contact with someone who has the c...,NaN,"No, I have not been tested",None,Sore throat,No,Stomach or abdominal pain,No,None,...,Feb. to early March 2020 or before,Never,Never,More recently,More recently,More recently,More recently,June/July 2020,Feb. to early March 2020 or before,Feb. to early March 2020 or before


In [4]:
# Select columns: Demographics, symptoms, history, and test results
data = data[data['TestResult'].isin(['Yes, and I tested negative','Yes, and I tested positive'])][['TestResult',
            'symptomsfirst','symptoms','symptomsresp','symptomsgastro','symptomsneuro'
            ,'symptomsinflamm'
            ,'age'
            ,'Ethnicity'
            ,'Race', 'gender'
            ,'symptomsconsresp', 'symptomsconsgastro', 'symptomsconsneuro'
            ,'symptomsconsinflamm'
            ]]

In [5]:
# convert the test result into numarical values (1: positive, 0: negative)
data['TestPositive'] = data['TestResult'].map({'Yes, and I tested positive': 1, 'Yes, and I tested negative': 0})

In [6]:
# Numbere of rows and columns
data.shape

(461, 16)

In [7]:
# convert history columns to numarical values (1: yes, 0: no or not sure)
data['IgnoreRespSymp']   = data['symptomsconsresp'].map(   {'Yes': 1, 'No': 0, 'Not sure' : 0, np.nan: 0})
data['IgnoreGastroSymp'] = data['symptomsconsgastro'].map( {'Yes': 1, 'No': 0, 'Not sure' : 0, np.nan: 0})
data['IgnoreNeuroSymp']  = data['symptomsconsneuro'].map(  {'Yes': 1, 'No': 0, 'Not sure' : 0, np.nan: 0})
data['IgnoreInflamSymp'] = data['symptomsconsinflamm'].map({'Yes': 1, 'No': 0, 'Not sure' : 0, np.nan: 0})

In [8]:
# compind all symptoms
data['symptoms_concat'] = data[['symptoms','symptomsresp','symptomsgastro','symptomsneuro',
                                'symptomsinflamm']].astype(str).apply(','.join, axis=1)

In [9]:
data = data.fillna('NaN')

In [10]:
# select columns 
data = data[['TestPositive',
             'symptomsfirst','symptoms_concat'
             , 'age'
             ,'Ethnicity'
             , 'Race', 'gender'
            , 'IgnoreRespSymp', 'IgnoreGastroSymp', 'IgnoreNeuroSymp', 'IgnoreInflamSymp'
            ]]

In [11]:
# Identify the list of symptom from "symptoms_concat" column
symptoms = list(set((','.join(data['symptoms_concat'].values.tolist())).split(",")))
symptoms = [sub.split(" (")[0] for sub in symptoms]

symptoms.remove('None')
symptoms.remove('None or I did not get tested')
symptoms.remove('nan')
    
len(symptoms)

29

In [12]:
# binarizing symptoms
for s in symptoms:
    data[s.lower()] = pd.np.where(data.symptoms_concat.str.contains(s), 1,0)

In [13]:
data.columns

Index(['TestPositive', 'symptomsfirst', 'symptoms_concat', 'age', 'Ethnicity',
       'Race', 'gender', 'IgnoreRespSymp', 'IgnoreGastroSymp',
       'IgnoreNeuroSymp', 'IgnoreInflamSymp', 'loss of taste', 'loss of smell',
       'new confusion', 'unexplained rashes anywhere else',
       'change in or loss of appetite',
       'any tingling/numbness/swelling in hands or feet', 'loss of balance',
       'joint or any other unexplained pain', 'fatigue', 'diarrhea',
       'runny nose', 'chest pain', 'muscle aches', 'pinkeye or conjunctivitis',
       'red or purple rash or lesions on your toes', 'nausea or vomiting',
       'chills', 'shortness of breath', 'slurred speech',
       'bluish lips or face', 'fever or feeling feverish', 'headaches',
       'excessive sweating', 'difficulty breathing',
       'unusual shivering or shaking', 'stomach or abdominal pain',
       'sore throat', 'wheezing', 'cough'],
      dtype='object')

In [14]:
data.shape

(461, 40)

In [15]:
# Convert age into 2 categories
data['Age 30 and over'] = np.where(data['age']>= 30, 1, 0)
data['Age 18 to 29'] = np.where(data['age']< 30, 1, 0)

data = data.drop(columns=[ 'age'])

In [16]:
data['Race'].value_counts()

White                                                               364
Black or African American                                            59
Asian                                                                21
American Indian or Alaska Native                                      5
Native Hawaiian or Other Pacific Islander                             4
Other, please specify:                                                3
White,Black or African American                                       2
White,Black or African American,American Indian or Alaska Native      1
White,Other, please specify:                                          1
White,Asian,Native Hawaiian or Other Pacific Islander                 1
Name: Race, dtype: int64

In [17]:
# Get dummies for arace, eethnicity, and gender
data = pd.get_dummies(data, columns=['Race', 'Ethnicity', 'gender'])
data = data.drop(columns=['Race_Other, please specify:', 'Ethnicity_Unknown', 'gender_Other', 
                          'Race_White,Other, please specify:'])

In [18]:
# select list of columns
cols = list(data.columns)
cols = [e for e in cols if e not in ('symptoms_concat', 'symptomsfirst', 'TestPositive')]

In [19]:
cols

['IgnoreRespSymp',
 'IgnoreGastroSymp',
 'IgnoreNeuroSymp',
 'IgnoreInflamSymp',
 'loss of taste',
 'loss of smell',
 'new confusion',
 'unexplained rashes anywhere else',
 'change in or loss of appetite',
 'any tingling/numbness/swelling in hands or feet',
 'loss of balance',
 'joint or any other unexplained pain',
 'fatigue',
 'diarrhea',
 'runny nose',
 'chest pain',
 'muscle aches',
 'pinkeye or conjunctivitis',
 'red or purple rash or lesions on your toes',
 'nausea or vomiting',
 'chills',
 'shortness of breath',
 'slurred speech',
 'bluish lips or face',
 'fever or feeling feverish',
 'headaches',
 'excessive sweating',
 'difficulty breathing',
 'unusual shivering or shaking',
 'stomach or abdominal pain',
 'sore throat',
 'wheezing',
 'cough',
 'Age 30 and over',
 'Age 18 to 29',
 'Race_American Indian or Alaska Native',
 'Race_Asian',
 'Race_Black or African American',
 'Race_Native Hawaiian or Other Pacific Islander',
 'Race_White',
 'Race_White,Asian,Native Hawaiian or Other

In [20]:
len(cols)

47

In [21]:
for i in range(len(symptoms)):
    symptoms[i] = symptoms[i].lower()

In [22]:
data = data.drop(columns=['symptomsfirst', 'symptoms_concat'])

In [23]:
data.columns = data.columns.str.replace("gender_Female", "Female")
data.columns = data.columns.str.replace("gender_Male", "Male")
data.columns = data.columns.str.replace("Ethnicity_Hispanic Latino", "Hispanic or Latino")
data.columns = data.columns.str.replace("Ethnicity_Non-Hispanic Latino", "Non Hispanic or Latino")
data.columns = data.columns.str.replace("Race_White", "Race White")
data.columns = data.columns.str.replace("Race_Black or African American", "Race Black or African American")
data.columns = data.columns.str.replace("Race_Native Hawaiian or Other Pacific Islander", "Race Native Hawaiian or Other Pacific Islander")
data.columns = data.columns.str.replace("Race_American Indian or Alaska Native", "Race American Indian or Alaska Native")
data.columns = data.columns.str.replace("Race_Asian", "Race Asian")
data.columns = data.columns.str.replace("Race White,Black or African American,American Indian or Alaska Native", "Mixed Race 1")
data.columns = data.columns.str.replace("Race White,Asian,Native Hawaiian or Other Pacific Islander", "Mixed Race 2")
data.columns = data.columns.str.replace("Race White,Black or African American", "Mixed Race 3")

In [24]:
data.columns = data.columns.str.replace("stomach or abdominal pain", "Abdominal pain")
data.columns = data.columns.str.replace("chest pain", "Chest pain")
data.columns = data.columns.str.replace("chills", "Chills")
data.columns = data.columns.str.replace("new confusion", "Confusion")
data.columns = data.columns.str.replace("cough", "Cough")
data.columns = data.columns.str.replace("diarrhea", "Diarrhea")
data.columns = data.columns.str.replace("excessive sweating", "Excessive sweating")
data.columns = data.columns.str.replace("fatigue", "Fatigue")
data.columns = data.columns.str.replace("fever or feeling feverish", "Fever")
data.columns = data.columns.str.replace("headaches", "Headaches")
data.columns = data.columns.str.replace("joint or any other unexplained pain", "Joint pain")
data.columns = data.columns.str.replace("change in or loss of appetite", "Loss of appetite")
data.columns = data.columns.str.replace("loss of balance", "Loss of balance")
data.columns = data.columns.str.replace("loss of smell", "Loss of smell")
data.columns = data.columns.str.replace("loss of taste", "Loss of taste")
data.columns = data.columns.str.replace("muscle aches", "Muscle aches")
data.columns = data.columns.str.replace("pinkeye or conjunctivitis", "Pinkeye")
data.columns = data.columns.str.replace("red or purple rash or lesions on your toes", "Red rash")
data.columns = data.columns.str.replace("runny nose", "Runny nose")
data.columns = data.columns.str.replace('unusual shivering or shaking', "Shivering")
data.columns = data.columns.str.replace("shortness of breath", "Shortness of breath")
data.columns = data.columns.str.replace('difficulty breathing', 'Difficulty breathing')
data.columns = data.columns.str.replace("sore throat", "Sore throat")
data.columns = data.columns.str.replace("unexplained rashes anywhere else", "Unexplained rash")
data.columns = data.columns.str.replace("nausea or vomiting", "Vomiting")
data.columns = data.columns.str.replace("wheezing", "Wheezing")
data.columns = data.columns.str.replace('bluish lips or face', 'Bluish lips or face')
data.columns = data.columns.str.replace('any tingling/numbness/swelling in hands or feet', 'Numbness')
data.columns = data.columns.str.replace('slurred speech', 'Slurred speech')
data.columns = data.columns.str.replace("IgnoreNeuroSymp", "History of neurological symptoms")
data.columns = data.columns.str.replace("IgnoreGastroSymp", "History of gastrointestinal symptoms")
data.columns = data.columns.str.replace("IgnoreRespSymp", "History of respiratory symptoms")
data.columns = data.columns.str.replace("IgnoreInflamSymp", "History of inflammatory symptoms")

In [25]:
data.columns  = [s.replace(' ','_') for s in data.columns]

In [26]:
data.columns.tolist()

['TestPositive',
 'History_of_respiratory_symptoms',
 'History_of_gastrointestinal_symptoms',
 'History_of_neurological_symptoms',
 'History_of_inflammatory_symptoms',
 'Loss_of_taste',
 'Loss_of_smell',
 'Confusion',
 'Unexplained_rash',
 'Loss_of_appetite',
 'Numbness',
 'Loss_of_balance',
 'Joint_pain',
 'Fatigue',
 'Diarrhea',
 'Runny_nose',
 'Chest_pain',
 'Muscle_aches',
 'Pinkeye',
 'Red_rash',
 'Vomiting',
 'Chills',
 'Shortness_of_breath',
 'Slurred_speech',
 'Bluish_lips_or_face',
 'Fever',
 'Headaches',
 'Excessive_sweating',
 'Difficulty_breathing',
 'Shivering',
 'Abdominal_pain',
 'Sore_throat',
 'Wheezing',
 'Cough',
 'Age_30_and_over',
 'Age_18_to_29',
 'Race_American_Indian_or_Alaska_Native',
 'Race_Asian',
 'Race_Black_or_African_American',
 'Race_Native_Hawaiian_or_Other_Pacific_Islander',
 'Race_White',
 'Mixed_Race_2',
 'Mixed_Race_3',
 'Mixed_Race_1',
 'Hispanic_or_Latino',
 'Non_Hispanic_or_Latino',
 'Female',
 'Male']

In [27]:
data.shape

(461, 48)

In [28]:
data = data.reset_index()
data = data.drop(columns=['index'])

In [29]:
data.to_csv("C:/Users/alexa/Downloads/preprocessed.csv",index=False)

In [30]:
# read list of index for eaach data (30 list of ids for training and 30 list of ids for testing)
ids = pd.read_csv("C:/Users/alexa/Downloads/preprocessed.csv")

In [31]:
# save the 30 training and testing data
for i in ids.columns: 
    index = [x for x in ids[i].values.tolist() if math.isnan(x) == False]
    d = data.loc[index]
    d.to_csv("C:/Users/alexa/Downloads/preprocessed.csv"+str(i), index=False)